# Census tract map of number of daily bus arrivals by CalEnviroScreen level

* 3 maps, one for each CalEnviroScreen level of low/medium/high
* Each map plots number of bus arrivals per 1k (colored as low/medium/high)
* One specific map for CalEnviroScreen high-need to show where there is zero service

In [ ]:
# In terminal: for ipyleaflet maps to render properly in notebook
!jupyter labextension install @jupyter-widgets/jupyterlab-manager
!jupyter labextension install jupyter-leaflet

In [ ]:
import branca
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

import prep_data
import map_utils

from IPython.display import Markdown, HTML

pd.set_option("display.max_rows", 20)

In [ ]:
df = prep_data.generate_stop_times_tract_data()

# Address small "numbers issue
df = df[(df.num_pop_jobs > 10)].reset_index(drop=True)

In [ ]:
sample_tracts = ["06001400100", "06001400600", "06001400700", "06001404501"]
#to_plot = df[df.Tract.isin(sample_tracts)]

# If equity_group==1, corresponds to lower CalEnviroScreen scores
# If equity_group==3, corresponds to higher CalEnviroScreen scores (worse off) -- focus on these

# Focus on high density areas, then map out low/med/high service
# For calenviroscreen worst ones, low/med/high service

## folium maps

In [ ]:
popup_dict = {
    "Tract": "Census Tract",
    "ZIP": "Zip Code",
    "County": "County",
    "City": "City",
    "equity_group": "CalEnviroScreen Group (3 is high need)", 
    "popjobdensity_group": "Pop & Job Density Group (3 is densest)",
    "num_arrivals": "# Daily Bus Arrivals",
    "stop_id": "# Bus Stops",
    "itp_id": "# Operators",
    "arrivals_per_1k_pj": "Arrivals per 1k",
    "arrivals_group_pj": "Arrivals Group (3 is highest)",
}

def grab_legend_thresholds(plot_col, arrivals_group):
    cut1 = df[(df[arrivals_group]==1) & (df[plot_col] > 0)][plot_col].min()
    cut2 = df[df[arrivals_group]==2][plot_col].min()
    cut3 = df[df[arrivals_group]==3][plot_col].min()

    MIN_VALUE = df[plot_col].min()
    MAX_VALUE = df[plot_col].max()
    
    colormap_cutoff = [cut1, cut2, cut3]
    
    return colormap_cutoff, MIN_VALUE, MAX_VALUE
    

cmap = plt.cm.get_cmap('viridis', 4)  

COLOR_LIST = []
for i in range(cmap.N):
    rgba = cmap(i)
    # rgb2hex accepts rgb or rgba
    hex_value = matplotlib.colors.rgb2hex(rgba)
    COLOR_LIST.append(hex_value)

COLOR_LIST.reverse()

# Maybe set the colors more explicitly
# Colors
gray = "#8D9191" 
light_green = "#6EB98F"
light_blue = "#1282A2"
navy = "#0A4C6A"

FIG_WIDTH = 800
FIG_HEIGHT = 1_000

#colorscale = branca.colormap.linear.Blues_09

In [ ]:
LOOP_GROUPS = {
    1: "Low", 
    2: "Med",
    3: "High",
}

plot_col = "arrivals_per_1k_pj"
arrivals_col = "arrivals_group_pj"
# For indexing cut-off, we need 1 less than the number of groups
# It goes from [vmin, cut1], [cut1, cut2], [cut2, cut3]
# Not sure where the open parantheses are [) or (]?
colormap_cutoff, MIN_VALUE, MAX_VALUE = grab_legend_thresholds(plot_col, 
                                                               arrivals_col)
print(colormap_cutoff)

colorscale = branca.colormap.LinearColormap(
                #colors=COLOR_LIST,
                colors=[gray, light_green, light_blue, navy], 
                index=colormap_cutoff,
                vmin=MIN_VALUE, vmax=colormap_cutoff[2] + 100,
)


for i, group_name in LOOP_GROUPS.items():
    to_plot = df[df.equity_group==i]
    # May need to adjust colorscale, how it shows up on map...
    fig = map_utils.make_folium_choropleth_map(to_plot, 
                                    plot_col = plot_col, 
                                    popup_dict = popup_dict, 
                                    tooltip_dict = popup_dict, 
                                    colorscale = colorscale, 
                                    fig_width = FIG_WIDTH, fig_height = FIG_HEIGHT, 
                                    zoom = map_utils.REGION_CENTROIDS["CA"][1], 
                                    centroid = map_utils.REGION_CENTROIDS["CA"][0],
                                    title=f"Bus Service per 1k for {group_name}-Need CalEnviroScreen Tracts",
                                   )
    fig.save(f"./img/arrivals_pc_{group_name.lower()}.html")
    #iframe = fig._repr_html_()
    #display(HTML(iframe))
    

In [ ]:
plot_col = "num_arrivals"
# For indexing cut-off, we need 1 less than the number of groups
# It goes from [vmin, cut1], [cut1, cut2], [cut2, cut3]
# Not sure where the open parantheses are [) or (]?
colormap_cutoff, MIN_VALUE, MAX_VALUE = grab_legend_thresholds(plot_col, arrivals_col)
print(colormap_cutoff)

colorscale = branca.colormap.LinearColormap(
                #colors=COLOR_LIST,
                colors=[gray, light_green, light_blue, navy], 
                index=colormap_cutoff,
                vmin=MIN_VALUE, vmax=colormap_cutoff[2] + 100,
)

for i, group_name in LOOP_GROUPS.items():
    to_plot = df[df.equity_group==i]
    display(colorscale)
    fig = map_utils.make_folium_choropleth_map(to_plot, 
                                    plot_col = plot_col, 
                                    popup_dict = popup_dict, 
                                    tooltip_dict = popup_dict, 
                                    colorscale = colorscale, 
                                    fig_width = FIG_WIDTH, fig_height = FIG_HEIGHT, 
                                    zoom = map_utils.REGION_CENTROIDS["CA"][1], 
                                    centroid = map_utils.REGION_CENTROIDS["CA"][0],                                    
                                    title=f"Bus Service for {group_name}-Need CalEnviroScreen Tracts",
                                   )
    
    fig.save(f"./img/arrivals_{group_name.lower()}.html")

In [ ]:
# Map for where there is zero service
#https://notebook.community/racu10/emapy/notebooks/Colormaps
popup_dict = {
    "Tract": "Census Tract",
    "ZIP": "Zip Code",
    "County": "County",
    "City": "City",
    "equity_group": "CalEnviroScreen Group (3 is worst)", 
    "popjobdensity_group": "Job/Population Density Group (3 is densest)",
}

plot_col = "popjobdensity_group"
to_plot = df[(df.equity_group==3) & (df.stop_id==0)]

colorscale = branca.colormap.StepColormap(
                colors=[light_green, light_blue, navy], 
                #index=[1, 2, 3],
                vmin=1, vmax=3,
)
display(colorscale)

fig = map_utils.make_folium_choropleth_map(to_plot, 
                                plot_col = plot_col, 
                                popup_dict = popup_dict, 
                                tooltip_dict = popup_dict, 
                                colorscale = colorscale, 
                                fig_width = FIG_WIDTH, fig_height = FIG_HEIGHT, 
                                zoom = map_utils.REGION_CENTROIDS["CA"][1], 
                                centroid = map_utils.REGION_CENTROIDS["CA"][0],
                                title=f"Zero Bus Service for High-Need CalEnviroScreen Tracts",
                               )
fig.save(f"./img/zero_service_high.html")

## ipyleaflet maps

In [ ]:
import ipyleaflet
import json

In [ ]:
def load_subset_data(df):
    geo_data = json.loads(df.set_index("Tract").to_json())

    # Take what we want to map and turn it into a dictionary
    # Can only include the key-value pair, the value you want to map, nothing more.
    gdf = dict(zip(df['Tract'].tolist(), df[plot_col].tolist()))
    
    return geo_data, gdf

In [ ]:
plot_col = "arrivals_per_1k_pj"
arrivals_col = "arrivals_group_pj"

colormap_cutoff, MIN_VALUE, MAX_VALUE = grab_legend_thresholds(plot_col, 
                                                               arrivals_col)


for i in range(1, 4):
    geo_data, choro_data = load_subset_data(df[df.equity_group==i]
                                            .drop(columns = ["arrivals_per_1k_p", 
                                                            "arrivals_per_1k_j"])
                                           )
        
    choropleth_dict = {
        "geo_data": geo_data,
        "choro_data": choro_data,
        "layer_name": "Daily Arrivals",
        "plot_col_name": "Arrivals per 1k",
        "MIN_VALUE": df[plot_col].min(),
        "MAX_VALUE": df[plot_col].quantile(0.98),
        "fig_width": '100%',
        "fig_height": '100%',
        "fig_min_width_px": '600px',
        "fig_min_height_px": '600px',
    }


    colorscale = branca.colormap.LinearColormap(
                    #colors=COLOR_LIST,
                    colors=[gray, light_green, light_blue, navy], 
                    index=colormap_cutoff,
                    vmin=MIN_VALUE, vmax=colormap_cutoff[2] + 100,
    )


    m = map_utils.make_ipyleaflet_choropleth_map(
        choropleth_dict = choropleth_dict,
        plot_col = plot_col,
        colorscale = colorscale, 
        zoom=map_utils.REGION_CENTROIDS["CA"][1], 
        centroid = map_utils.REGION_CENTROIDS["CA"][0], 
    )

    display(m)